In [ ]:
import numpy as np
import itertools
import traceback
import random as rd
from matplotlib import pyplot as plt
#idk if I'll need all of these but they sure do help

num_players = 2
pop = 10000
moves = list(range(7)) #move of 3 represents the decision to select a Type 3 square as a pickup location
collect_payoff = 50
handover_payoff = 50
delivery_payoff = 50

In [ ]:
def payoff(my_move:int, their_move:int, package_collector = False):
    if package_collector: #if collector (ie. Player 1), costs are easy
        if my_move + their_move == 6:
            return (collect_payoff + handover_payoff - (2.3 + my_move)**2)
        return (collect_payoff - (2.3 + my_move)**2)
    #if not collector, use variable expected cost based on avg. distance
    costs = {0:2.3, 1:1.3, 2:0.77, 3:0.6, 4:0.77, 5:1.3, 6:2.3}
    if my_move + their_move == 6:
        return (handover_payoff + delivery_payoff - (costs[my_move] + my_move)**2)
    return -(costs[my_move] + my_move)**2

def game(agent_1, agent_2):
    if agent_1.collector != agent_2.collector:
        agent_1_payoff = payoff(agent_1.strategy, agent_2.strategy, agent_1.collector)
        agent_2_payoff = payoff(agent_2.strategy, agent_1.strategy, agent_2.collector)
        agent_1.per_round_payoff = agent_1_payoff
        agent_2.per_round_payoff = agent_2_payoff

class Agent():
    def __init__(self, aid, strategy, collector=True) -> None:
        self.aid = aid
        self.strategy = strategy
        self.collector = collector
        self.per_round_payoff = 0

In [ ]:
class Replicator():
    def __init__(self, population:int = 1000, epochs:int = 100) -> None:
        self.population = population
        self.p1_proportions = [1/7, 1/7, 1/7, 1/7, 1/7, 1/7, 1/7]
        self.p2_proportions = [1/7, 1/7, 1/7, 1/7, 1/7, 1/7, 1/7]
        self.agents = self.setup_agents()
        self.epochs = epochs
        self.p1_history = {0:[], 1:[], 2:[], 3:[], 4:[], 5:[], 6:[]}
        self.p2_history = {0:[], 1:[], 2:[], 3:[], 4:[], 5:[], 6:[]}

    def setup_agents(self):
        agents = []
        for i in range(self.population):
            collector = np.random.choice([True, False])
            if collector:
                s = np.random.choice(moves, p=self.p1_proportions)
            else:
                s = np.random.choice(moves, p=self.p2_proportions)
            agents.append(Agent(i,s,collector))
        return agents
    
    def step(self):
        pairs = list(itertools.combinations(self.agents, 2))
        for pair in pairs:
            game(*pair)

    def update_proportions(self):
        p1_proportions = np.zeros(7)
        p2_proportions = np.zeros(7)
        for agent in self.agents:
            if agent.collector:
                p1_proportions[agent.strategy] += agent.per_round_payoff
                agent.per_round_payoff = 0
            else:
                p2_proportions[agent.strategy] += agent.per_round_payoff
                agent.per_round_payoff = 0
        p1_proportions += np.abs(np.min(p1_proportions))
        p2_proportions += np.abs(np.min(p2_proportions))
        self.p1_proportions = p1_proportions/np.sum(p1_proportions)
        self.p2_proportions = p2_proportions/np.sum(p2_proportions)
        print(f"P1 proportions: {self.p1_proportions}")
        print(f"P2 proportions: {self.p2_proportions}")

    def log(self):
        for i in range(7):
            self.p1_history[i].append(self.p1_proportions[i])
            self.p2_history[i].append(self.p2_proportions[i])

    def run(self):
        self.log()
        for epoch in range(self.epochs):
            print(f"Epoch: {epoch}")
            self.step()
            self.update_proportions()
            self.agents = self.setup_agents()
            self.log()
        
    def visualise(self):
        fig, ax = plt.subplots(1,2)
        x = np.linspace(0,self.epochs+1, self.epochs+1)
        for i in range(7):
            ln = ax[0].plot(x, self.p1_history[i], label=f"Proportion of {i}")
        for i in range(7):
            ln = ax[1].plot(x, self.p2_history[i], label=f"Proportion of {i}")
        ax[0].legend()
        ax[1].legend()
        plt.show()



In [ ]:
kate = Replicator(epochs=20)
kate.run()
kate.visualise()